# IMPORT

In [1]:
import numpy as np
import pandas as pd

In [2]:
# custom username by pwd
import os
pwd = os.getcwd()
username = pwd[9:pwd.find('\\',9)]

In [3]:
def isEnglish(s):
  try:
    s.encode(encoding='utf-8').decode('ascii')
  except UnicodeDecodeError:
    return False
  else:
    return True

In [4]:
import difflib

def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.90):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff)
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None

In [5]:
input_path = 'C:\\Users\\'+ username +'\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\Data\\ปี 2567 Q1\\'
result_path = 'C:\\Users\\'+ username +'\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\3. รวมศูนย์\\RESULT\\'

# input_path = r'C:\Users\tharaphonn\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\Data\ปี 2566 Q3/'
# result_path = r'C:\Users\tharaphonn\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\3. รวมศูนย์\RESULT/'

In [6]:
def clean_ecs_name(fname , lname):
  name1 = np.nan
  name2 = np.nan
  if pd.isna(fname) == False:
    if fname == '-':
      fname = np.nan

  if pd.isna(lname) == False:
    if lname == '-':
      lname = np.nan

  if (pd.isna(fname) == False) and (pd.isna(lname) == False):
    if isEnglish(fname) == isEnglish(lname):
      name1 = fname + lname
      name2 = np.nan
    else:
      name1 = fname
      name2 = lname
  elif (pd.isna(fname) == False) and (pd.isna(lname) == True):
    name1 = fname
    name2 = np.nan
  elif (pd.isna(fname) == True) and (pd.isna(lname) == False):
    name1 = lname
    name2 = np.nan
  elif (pd.isna(fname) == False) and (pd.isna(lname) == False):
    name1 = np.nan
    name2 = np.nan
  return name1 , name2

In [7]:
def clean_string(x):
  if pd.isna(x) == False:
    x = x.lower()
    x = x.strip()
    remove_list = [' ','ด็อกเตอร์','ดร.','ว่าที่','พล.อ.','พล.ท.','พล.ต.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต','ส.อ.',
                           'ส.ท.','ส.ต.',
                          'พลฯ','นนร.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','น.อ.','ร.น.','น.ท.','น.ต.','ร.อ.','ร.ท.','ร.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.',
                          'จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','นนอ.','นจอ.',
                          'พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี',
                           'mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/' ,
                           'นาย','นางสาว' ,'นาง' , 'x0000' , '_' , 'และหรือย่อย']
    for i in range(len(remove_list)):
      x = x.replace(remove_list[i],'')
  return x

# INPUT

In [8]:
#@title ข้อมูลรวมศูนย์บนระบบ CRM
centralize = 'รวมศูนย์_20240402.xlsx' #@param {type:'string'}
df_cen = pd.read_excel( input_path + centralize , dtype = str)

In [9]:
#@title ข้อมูลกรรมการ/ผู้ถือหุ้น ENLITE
enlite = 'ENLITE_6365_Q1_67_20240424.xlsx' #@param {type:'string'}
df_enl = pd.read_excel( input_path + enlite , dtype = str)

In [10]:
#@title ข้อมูล Outstanding
outstanding = '03.วงเงินและยอดคงค้าง_มี.ค.2567.xlsx' #@param {type:'string'}
df_out = pd.read_excel(input_path + outstanding , dtype = str)

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [11]:
#@title ข้อมูล รหัส และ เลขนิติบุคคลของลูกค้า
id_cuscode = 'รหัส_เลขนิติ_Q1_67.xlsx' #@param {type:'string'}
df_pm = pd.read_excel( input_path + id_cuscode, dtype= str)

In [12]:
#@title ชื่อไฟล์ OUTPUT
file_name = 'รวมศูนย์_กรรมการผู้ถือหุ้น_q1_67_jupyter.xlsx' #@param {type:'string'}

In [13]:
#@title ใส่ขอบเขตการตรวจ
start_date = '2024-01-01' #@param {type:"date"}
end_date = '2024-03-31' #@param {type:"date"}
start_date = pd.to_datetime( start_date, format = '%Y-%m-%d')
end_date = pd.to_datetime(end_date  , format = '%Y-%m-%d')

# CODE

## Chose Customer and Clean Outstanding File

In [14]:
df_out1 = df_out[['รหัสลูกค้า'  ,'ผู้ดูแลลูกค้า_(แยกส่วน)' , 'ผู้ดูแลลูกค้า_(ตามฝ่าย)' , 'วันที่อนุมัติ',
                           'วันที่ทบทวนล่าสุด']].drop_duplicates()

In [15]:
def clean_date_out(a):
  if pd.isna(a) == False:
    if a[6:8] == '26':
      a = a.replace('265' , '256')
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '25':
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '20' or a[6:8] == '19':
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:7] == '3':
      a = a[:6] + str(int(a[6:]) -2*543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '00':
      a = np.nan
  return a

In [16]:
date_col = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด']
for i in range(len(date_col)):
  print(date_col[i])
  df_out1[date_col[i]] = df_out1[date_col[i]].apply(lambda x : clean_date_out(x))
 # df_out1[date_col[i]] = pd.to_datetime( df_out1[date_col[i]] , format = '%d/%m/%Y')


วันที่อนุมัติ
วันที่ทบทวนล่าสุด


In [17]:
df_out1 = df_out1.dropna(subset = ['ผู้ดูแลลูกค้า_(แยกส่วน)'])

In [18]:
def check_date_interval(a , b , start_date , end_date):
  c = 0
  if pd.isna(a) == False:
    if (a >= start_date) and (a<= end_date):
      c+=1
  if pd.isna(b) == False:
    if (b >= start_date) and (b <= end_date):
      c +=1
  return c

In [19]:
df_out1['is_in'] = df_out1.apply(lambda x : check_date_interval(x['วันที่อนุมัติ'] , x['วันที่ทบทวนล่าสุด'] , start_date , end_date) , axis = 1)

In [20]:
df_out1.head()

,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),วันที่อนุมัติ,วันที่ทบทวนล่าสุด,is_in
0,G009336,สาขาพระราม 4,สาขาพระราม 4,2021-04-30,2023-08-02,0
1,G009336,สาขาพระราม 4,สาขาพระราม 4,2016-09-06,2023-08-02,0
2,G009336,สาขาพระราม 4,สาขาพระราม 4,2022-09-23,2023-08-02,0
3,G009348,ส่วนคดี,ฝ่ายบริหารหนี้และนิติกรรม,2015-04-09,2023-11-30,0
5,G009525,ส่วนอุตสาหกรรม 3.1,ฝ่ายกลุ่มอุตสาหกรรม 3,2010-02-12,2023-12-22,0


In [21]:
# Clean ข้อมูลให้เหลือ 1 บรรทัดก่อนของข้อมูล outstanding
cus_list = df_out1['รหัสลูกค้า'].unique().tolist()
df_out2 = pd.DataFrame()
for i in range(len(cus_list)):
  df_t = df_out1[df_out1['รหัสลูกค้า'] == cus_list[i]]

  date1_min = min(list(df_t['วันที่อนุมัติ']))
  date1_max = max(list(df_t['วันที่อนุมัติ']))

  #date2_min = min(list(df_t['วันที่ทบทวนล่าสุด']))
  date2_max = min(list(df_t['วันที่ทบทวนล่าสุด']))

  df_t1 = df_t.head(1)
  df_t1 = df_t1.drop(columns = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด'])
  df_t1['วันที่อนุมัติครั้งแรก'] = date1_min
  df_t1['วันที่อนุมัติล่าสุด'] = date1_max
  #df_t1['วันที่ทบทวนครั้งแรก'] = date2_min
  df_t1['วันที่ทบทวนล่าสุด'] = date2_max

  df_out2 = pd.concat([df_out2 , df_t1])

In [22]:
df_cusin = df_out1[df_out1['is_in'] != 0][['รหัสลูกค้า']].drop_duplicates()

In [23]:
df_out3 = pd.merge(df_out2 , df_cusin , on = 'รหัสลูกค้า')

In [24]:
print(len(df_out1) , len(df_out2) , len(df_out3))

8305 5089 1945


In [25]:
df_out4 = pd.merge(df_out3 , df_pm , on = 'รหัสลูกค้า' , how  ='left')

In [26]:
#df_out4 = df_out4.drop(columns = ['is_in'])
df_out4.head()

,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),is_in,วันที่อนุมัติครั้งแรก,วันที่อนุมัติล่าสุด,วันที่ทบทวนล่าสุด,CUCTAXCS
0,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105545080243
1,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105538144240
2,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105548072225
3,G020405,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2013-02-22,2021-04-30,2024-03-22,0105562130181
4,G020774,สาขาพระราม 4,สาขาพระราม 4,1,2012-07-26,2022-02-01,2024-03-12,0105530030512


## Clean ENLITE File

In [27]:
#df_enl1 = df_enl[[ 'เลขทะเบียนนิติบุคคล (13 หลัก)',
#       'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ' , 'อำนาจกรรมการ' ,'% ที่ถือหุ้นของ Held By 1', '% ที่ถือหุ้นของ Held By 2',
#       '% ที่ถือหุ้นของ Held By 3', '% ที่ถือหุ้นของ Held By 4',
#       '% ที่ถือหุ้นของ Held By 5', '% ที่ถือหุ้นของ Held By 6',
#       '% ที่ถือหุ้นของ Held By 7', '% ที่ถือหุ้นของ Held By 8',
#       '% ที่ถือหุ้นของ Held By 9', '% ที่ถือหุ้นของ Held By 10',
#       'ชื่อกรรมการ 1', 'ชื่อกรรมการ 2', 'ชื่อกรรมการ 3', 'ชื่อกรรมการ 4',
#       'ชื่อกรรมการ 5', 'ชื่อกรรมการ 6', 'ชื่อกรรมการ 7', 'ชื่อกรรมการ 8',
#       'ชื่อกรรมการ 9', 'ชื่อกรรมการ 10']]

In [28]:
df_enl1 = df_enl[[ 'เลขทะเบียนนิติบุคคล (13 หลัก)',
       'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ' , 'อำนาจกรรมการ' ,'% ที่ถือหุ้นของ Held By 1', '% ที่ถือหุ้นของ Held By 2',
       '% ที่ถือหุ้นของ Held By 3', '% ที่ถือหุ้นของ Held By 4',
       '% ที่ถือหุ้นของ Held By 5',
       'ชื่อกรรมการ 1', 'ชื่อกรรมการ 2', 'ชื่อกรรมการ 3', 'ชื่อกรรมการ 4',
       'ชื่อกรรมการ 5']]

In [29]:
share_col = []
director_col = []
col_list =  df_enl1.columns.tolist()
for i in range(len(col_list)):
  if 'ชื่อกรรมการ' in col_list[i]:
    director_col.append(col_list[i])
  elif 'ที่ถือหุ้นของ' in col_list[i]:
    share_col.append(col_list[i])

In [30]:
id_list = df_enl1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()

In [31]:
df_enl2 = pd.DataFrame()
for i in range(len(id_list)):
  df_t = df_enl1[df_enl1['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id_list[i]]
  df_t.reset_index(drop = True , inplace = True)
  director_name = []
  for m1 in range(len(director_col)):
    if df_t.loc[0 , director_col[m1]] != '-':
      director_name.append(df_t.loc[0 , director_col[m1]])
  share_name = []
  for m2 in range(len(share_col)):
    if df_t.loc[0,share_col[m2]] != '-':
      share_name.append(df_t.loc[0,share_col[m2]])

  df_t1 = df_t.head(1)
  for m3 in range(len(director_col)):
    df_t1 = df_t1.drop(columns = [director_col[m3]])
  for m4 in range(len(share_col)):
    df_t1 = df_t1.drop(columns = [share_col[m4]])

  df_t1_1 = pd.DataFrame()
  df_t1_1['ชื่อผู้เกี่ยวข้อง'] = director_name
  df_t1_1['ความเกี่ยวข้อง'] = 'กรรมการ'


  df_t1_2 = pd.DataFrame()
  df_t1_2['ชื่อผู้เกี่ยวข้อง'] = share_name
  df_t1_2['ความเกี่ยวข้อง'] = 'ผู้ถือหุ้น'

  df_t2 = pd.concat([df_t1_1 , df_t1_2])
  df_t2.reset_index(drop = True , inplace = True)

  fill_col = ['เลขทะเบียนนิติบุคคล (13 หลัก)', 'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ' , 'อำนาจกรรมการ' ]
  for n in range(len(fill_col)):
    df_t2[fill_col[n]] = df_t.loc[0,fill_col[n]]

  df_enl2 = pd.concat([df_enl2 , df_t2])

In [32]:
def clean_name_enl(a):
  if pd.isna(a) == False:
    if a == '-':
      a = np.nan
  return a

In [33]:
df_enl2['ชื่อผู้เกี่ยวข้อง'] = df_enl2['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : clean_name_enl(x))

In [34]:
df_enl3 = df_enl2.dropna(subset = ['ชื่อผู้เกี่ยวข้อง'])

In [35]:
print(len(df_enl2) , len(df_enl3))

28584 28584


In [36]:
df_enl3.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ
0,จุฑารัตน์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...
1,สมบูรณ์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...
2,จุฑารัตน์ จรัสภิญโญวงศ์ (70.00%),ผู้ถือหุ้น,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...
3,เกรียงไกร จรัสภิญโญวงศ์ (10.00%),ผู้ถือหุ้น,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...
4,สมบูรณ์ จรัสภิญโญวงศ์ (10.00%),ผู้ถือหุ้น,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...


In [37]:
def split_share_percent(a):
  b1 = np.nan
  b2 = np.nan
  if pd.isna(a) == False:
    b = a.rsplit(' ' , 1)
    b1 = b[0]
    b1 = b1.lower()
    b2 = b[1]
    clean_str1 = ['(' , ')' , '%' , ' ']
    for i in range(len(clean_str1)):
      b2 = b2.replace(clean_str1[i] , '')
    b2 = float(b2)
    clean_str2 = [' ','ด็อกเตอร์','ดร.','ว่าที่','พล.อ.','พล.ท.','พล.ต.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต','ส.อ.',
                           'ส.ท.','ส.ต.',
                          'พลฯ','นนร.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','น.อ.','ร.น.','น.ท.','น.ต.','ร.อ.','ร.ท.','ร.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.',
                          'จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','นนอ.','นจอ.',
                          'พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี',
                           'mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/']
    for i in range(len(clean_str2)):
      b1 = b1.replace(clean_str2[i] , '')
  b1 = b1.replace('ฏ' , 'ฎ')
  return b1 , b2

In [38]:
df_s = df_enl3[df_enl3['ความเกี่ยวข้อง'] == 'ผู้ถือหุ้น']
df_d = df_enl3[df_enl3['ความเกี่ยวข้อง'] == 'กรรมการ']

In [39]:
df_s['Name_clean'] , df_s['เปอร์เซ็นผู้ถือหุ้น'] = zip(*df_s['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : split_share_percent(x)))

C:\Users\tharaphonn\AppData\Local\Temp\ipykernel_23224\545980860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['Name_clean'] , df_s['เปอร์เซ็นผู้ถือหุ้น'] = zip(*df_s['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : split_share_percent(x)))
C:\Users\tharaphonn\AppData\Local\Temp\ipykernel_23224\545980860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['Name_clean'] , df_s['เปอร์เซ็นผู้ถือหุ้น'] = zip(*df_s['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : split_share_percent(x)))


In [40]:
def clean_director(a):
  if pd.isna(a) == False:
    clean_str2 = [' ','ด็อกเตอร์','ดร.','ว่าที่','พล.อ.','พล.ท.','พล.ต.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต','ส.อ.',
                           'ส.ท.','ส.ต.',
                          'พลฯ','นนร.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','น.อ.','ร.น.','น.ท.','น.ต.','ร.อ.','ร.ท.','ร.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.',
                          'จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','นนอ.','นจอ.',
                          'พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี',
                           'mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/']

  for i in range(len(clean_str2)):
    a = a.replace(clean_str2[i] , '')
  a = a.replace('ฏ' , 'ฎ')

  return a

In [41]:
df_d['Name_clean'] = df_d['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : clean_director(x))

C:\Users\tharaphonn\AppData\Local\Temp\ipykernel_23224\1359428930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d['Name_clean'] = df_d['ชื่อผู้เกี่ยวข้อง'].apply(lambda x : clean_director(x))


In [42]:
df_d.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean
0,จุฑารัตน์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...,จุฑารัตน์จรัสภิญโญวงศ์
1,สมบูรณ์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...,สมบูรณ์จรัสภิญโญวงศ์
0,ปรัชญา อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ปรัชญาอิทธิรส
1,ภาวิน อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ภาวินอิทธิรส
0,ณัฏฐ์มาติกา ลลิดาสุตานันท์,กรรมการ,0115549002107,บริษัท 2 เอ็มเอ็น จำกัด,"2 MN CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ณัฎฐ์มาติกาลลิดาสุตานันท์


In [43]:
def clean_power(x):
  if pd.isna(x) == False:
    x = x.lower()
    x = x.strip()
    x = x.replace(' ','')
    str_level_list = ['ด็อกเตอร์','ดร.','ว่าที่','พล\.อ\.','พล\.ท\.','พล\.ต\.','พ\.อ\.','พ\.ท\.','พ\.ต\.','ร\.อ\.','ร\.ท\.','ร\.ต\.','จ\.ส\.อ.','จ\.ส\.ท.','จ\.ส\.ต','ส\.อ\.',
                           'ส\.ท\.','ส\.ต\.',
                          'พลฯ','นนร\.','พล\.ร\.อ\.','พล\.ร\.ท\.','พล\.ร\.ต.','น\.อ\.','ร\.น\.','น\.ท\.','น\.ต\.','ร\.อ\.','ร\.ท\.','ร\.ต\.','พ\.จ\.อ.','พ\.จ\.ท.','พ\.จ\.ต.',
                          'จ\.อ\.','จ\.ท\.','จ\.ต\.','พล\.อ\.อ.','พล\.อ\.ท\.','พล\.อ\.ต\.','พ\.อ\.อ\.','พ\.อ\.ท\.','พ\.อ\.ต\.','นนอ\.','นจอ\.',
                          'พล\.ต\.อ\.','พล\.ต\.ท\.','พล\.ต\.ต\.','พ\.ต\.อ\.','พ\.ต\.ท\.','พ\.ต\.ต\.','ร\.ต\.อ\.','ร\.ต\.ท\.','ร\.ต\.ต\.','ด\.ต\.','จ\.ส\.ต\.','ส\.ต\.อ\.','ส\.ต\.ท\.','ส\.ต\.ต\.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี','mr.','mrs.','ms.']
    det_str = ['mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/']
    for i in range(len(str_level_list)):
      x = x.replace(str_level_list[i],'นาย')
    for i in range(len(det_str)):
      x = x.replace(det_str[i],'')
  x = x.replace('ฏ','ฎ')

  return x

In [44]:
df_d['อำนาจกรรมการ_clean'] = df_d['อำนาจกรรมการ'].apply(lambda x : clean_power(x))

C:\Users\tharaphonn\AppData\Local\Temp\ipykernel_23224\1870004402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d['อำนาจกรรมการ_clean'] = df_d['อำนาจกรรมการ'].apply(lambda x : clean_power(x))


In [45]:
df_d.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจกรรมการ_clean
0,จุฑารัตน์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...,จุฑารัตน์จรัสภิญโญวงศ์,นางสาวจุฑารัตน์จรัสภิญโญวงศ์หรือนางสมบูรณ์จรัส...
1,สมบูรณ์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...,สมบูรณ์จรัสภิญโญวงศ์,นางสาวจุฑารัตน์จรัสภิญโญวงศ์หรือนางสมบูรณ์จรัส...
0,ปรัชญา อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ปรัชญาอิทธิรส,กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ
1,ภาวิน อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ภาวินอิทธิรส,กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ
0,ณัฏฐ์มาติกา ลลิดาสุตานันท์,กรรมการ,0115549002107,บริษัท 2 เอ็มเอ็น จำกัด,"2 MN CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ณัฎฐ์มาติกาลลิดาสุตานันท์,กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ


In [46]:
def check_director(a,b):
  if pd.isna(b) == True:
    c = np.nan
  elif a in b:
    c = 'มีอำนาจลงนาม'
  elif '(1)' in b:
    c = np.nan
  elif ( ('นาย' in b) or ('นาง' in b) ) and a not in b:
    c = 'ไม่มีอำนาจลงนาม'
  elif b == 'ไม่มี':
    c = np.nan
  else:
    c = 'มีอำนาจลงนาม'
  return c

In [47]:
df_d['อำนาจลงนาม'] = df_d.apply(lambda x : check_director(x['Name_clean'],x['อำนาจกรรมการ_clean']) , axis = 1)

C:\Users\tharaphonn\AppData\Local\Temp\ipykernel_23224\2444726065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d['อำนาจลงนาม'] = df_d.apply(lambda x : check_director(x['Name_clean'],x['อำนาจกรรมการ_clean']) , axis = 1)


In [48]:
df_d.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจกรรมการ_clean,อำนาจลงนาม
0,จุฑารัตน์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...,จุฑารัตน์จรัสภิญโญวงศ์,นางสาวจุฑารัตน์จรัสภิญโญวงศ์หรือนางสมบูรณ์จรัส...,มีอำนาจลงนาม
1,สมบูรณ์ จรัสภิญโญวงศ์,กรรมการ,0905545002431,บริษัท 11 วงจร เทรดดิ้ง แอนด์ โคลด์ สตอเรจ จำกัด,"11 WONGJORN TRADING & COLD STORAGE CO.,LTD.",นางสาวจุฑารัตน์ จรัสภิญโญวงศ์ หรือ นางสมบูรณ์ ...,สมบูรณ์จรัสภิญโญวงศ์,นางสาวจุฑารัตน์จรัสภิญโญวงศ์หรือนางสมบูรณ์จรัส...,มีอำนาจลงนาม
0,ปรัชญา อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ปรัชญาอิทธิรส,กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ,มีอำนาจลงนาม
1,ภาวิน อิทธิรส,กรรมการ,0505556011511,บริษัท 118 ทริปเปิ้ล พี จำกัด,"118 TRIPLE P CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ภาวินอิทธิรส,กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ,มีอำนาจลงนาม
0,ณัฏฐ์มาติกา ลลิดาสุตานันท์,กรรมการ,0115549002107,บริษัท 2 เอ็มเอ็น จำกัด,"2 MN CO.,LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,ณัฎฐ์มาติกาลลิดาสุตานันท์,กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ,มีอำนาจลงนาม


In [49]:
df_s1 = df_s[df_s['เปอร์เซ็นผู้ถือหุ้น'] >= 20]
# df_s1 = df_s.copy()

In [50]:
df_d1 = df_d[df_d['อำนาจลงนาม'] != 'ไม่มีอำนาจลงนาม']
#df_d1 = df_d.copy()

In [51]:
df_en1 = pd.concat([df_d1,df_s1])

In [52]:
print(len(df_en1) , len(df_d1) , len(df_s1))

16911 9106 7805


In [53]:
df_en1 =df_en1.drop(columns = ['อำนาจกรรมการ_clean'])

In [54]:
df_en1 = df_en1.sort_values(['เลขทะเบียนนิติบุคคล (13 หลัก)' , 'ความเกี่ยวข้อง'])

In [55]:
df_en1.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น
0,พรวิไล พันธุสิงห์,กรรมการ,0103512009670,ห้างหุ้นส่วนจำกัด เอ๊กโคคอนสตรั๊กชั่น อินยิเนี...,ECHO CONSTRUCTION ENGINEERING LIMITED PARTNERSHIP,ไม่มี,พรวิไลพันธุสิงห์,NaN,NaN
1,พรวิไล พันธุสิงห์ (99.80%),ผู้ถือหุ้น,0103512009670,ห้างหุ้นส่วนจำกัด เอ๊กโคคอนสตรั๊กชั่น อินยิเนี...,ECHO CONSTRUCTION ENGINEERING LIMITED PARTNERSHIP,ไม่มี,พรวิไลพันธุสิงห์,NaN,99.80
0,จาตุรงค์ จรัสตระกูล,กรรมการ,0103517003998,ห้างหุ้นส่วนจำกัด จัตุรงค์คูลลิ่ง,CHATURONG COOLING LIMITED PARTNERSHIP,ไม่มี,จาตุรงค์จรัสตระกูล,NaN,NaN
1,จาตุรงค์ จรัสตระกูล (93.41%),ผู้ถือหุ้น,0103517003998,ห้างหุ้นส่วนจำกัด จัตุรงค์คูลลิ่ง,CHATURONG COOLING LIMITED PARTNERSHIP,ไม่มี,จาตุรงค์จรัสตระกูล,NaN,93.41
0,ไชย เตชะสมบูรณากิจ,กรรมการ,0103521001856,ห้างหุ้นส่วนจำกัด โรงงานประดิษฐ์กรณ์เท็กซ์ไทล์,P.D.K.TEXTILE FACTORY LIMITED PARTNERSHIP,การทำนิติกรรมใด ๆ เพื่อให้มีผลผูกพันห้างหุ้นส่...,ไชยเตชะสมบูรณากิจ,มีอำนาจลงนาม,NaN


In [56]:
df_en2 = pd.DataFrame()
cus_id = df_en1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()
for i in range(len(cus_id)):
  df_t = df_en1[df_en1['เลขทะเบียนนิติบุคคล (13 หลัก)'] == cus_id[i]].copy()
  name_list = df_t['Name_clean'].unique().tolist()
  for j in range(len(name_list)):
    df_t1 = df_t[df_t['Name_clean'] == name_list[j]]
    df_t1.reset_index(drop = True , inplace = True)
    a1 = df_t1.loc[0,'ความเกี่ยวข้อง']
    a2 = df_t1.loc[0,'เปอร์เซ็นผู้ถือหุ้น']
    if len(df_t1) > 1:
      a2 = df_t1.loc[1,'เปอร์เซ็นผู้ถือหุ้น']
    for k in range(1 , len(df_t1)):
      a1 = a1 +'/' +df_t1.loc[k,'ความเกี่ยวข้อง']
    df_t1.loc[0,'ความเกี่ยวข้อง'] = a1
    df_t1.loc[0 , 'เปอร์เซ็นผู้ถือหุ้น'] = a2
    df_t2 = df_t1.head(1)
    df_en2 = pd.concat([df_en2 , df_t2])

In [57]:
df_en2.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น
0,พรวิไล พันธุสิงห์,กรรมการ/ผู้ถือหุ้น,0103512009670,ห้างหุ้นส่วนจำกัด เอ๊กโคคอนสตรั๊กชั่น อินยิเนี...,ECHO CONSTRUCTION ENGINEERING LIMITED PARTNERSHIP,ไม่มี,พรวิไลพันธุสิงห์,NaN,99.80
0,จาตุรงค์ จรัสตระกูล,กรรมการ/ผู้ถือหุ้น,0103517003998,ห้างหุ้นส่วนจำกัด จัตุรงค์คูลลิ่ง,CHATURONG COOLING LIMITED PARTNERSHIP,ไม่มี,จาตุรงค์จรัสตระกูล,NaN,93.41
0,ไชย เตชะสมบูรณากิจ,กรรมการ/ผู้ถือหุ้น,0103521001856,ห้างหุ้นส่วนจำกัด โรงงานประดิษฐ์กรณ์เท็กซ์ไทล์,P.D.K.TEXTILE FACTORY LIMITED PARTNERSHIP,การทำนิติกรรมใด ๆ เพื่อให้มีผลผูกพันห้างหุ้นส่...,ไชยเตชะสมบูรณากิจ,มีอำนาจลงนาม,40.00
0,จุรี เตชะสมบูรณากิจ,กรรมการ,0103521001856,ห้างหุ้นส่วนจำกัด โรงงานประดิษฐ์กรณ์เท็กซ์ไทล์,P.D.K.TEXTILE FACTORY LIMITED PARTNERSHIP,การทำนิติกรรมใด ๆ เพื่อให้มีผลผูกพันห้างหุ้นส่...,จุรีเตชะสมบูรณากิจ,มีอำนาจลงนาม,NaN
0,ชาตรี เตชะสมบูรณากิจ,กรรมการ/ผู้ถือหุ้น,0103521001856,ห้างหุ้นส่วนจำกัด โรงงานประดิษฐ์กรณ์เท็กซ์ไทล์,P.D.K.TEXTILE FACTORY LIMITED PARTNERSHIP,การทำนิติกรรมใด ๆ เพื่อให้มีผลผูกพันห้างหุ้นส่...,ชาตรีเตชะสมบูรณากิจ,มีอำนาจลงนาม,30.00


In [58]:
df_en21 = df_en2.copy()

In [59]:
id_list = df_en2['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique().tolist()
df_en2 = pd.DataFrame()
fill_col = ['ชื่อนิติบุคคลจดทะเบียนภาษาไทย' , 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ']
for i in range(len(id_list)):
  df_t = df_en21[df_en21['เลขทะเบียนนิติบุคคล (13 หลัก)'] == id_list[i]]
  df_t = df_t.sort_values(['ชื่อนิติบุคคลจดทะเบียนภาษาไทย'])
  df_t.reset_index(drop = True , inplace = True)
  if pd.isna(df_t.loc[0,'ชื่อนิติบุคคลจดทะเบียนภาษาไทย']) == False:
    for j in range(len(fill_col)):
      df_t[fill_col[j]] = df_t[fill_col[j]].fillna(df_t.loc[0,fill_col[j]])
  df_en2 = pd.concat([df_en2 , df_t])

In [60]:
print(len(df_en1) , len(df_en21) , len(df_en2))

16911 11513 11513


In [61]:
print(len(df_en1['Name_clean'].unique()) , len(df_en2['Name_clean'].unique()))

9998 9998


In [62]:
print(len(df_en1['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique()) , len(df_en2['เลขทะเบียนนิติบุคคล (13 หลัก)'].unique()))

4994 4994


In [63]:
df_en2.reset_index(drop = True , inplace = True)

In [64]:
len(df_en2)

11513

In [65]:
df_en2.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น
0,พรวิไล พันธุสิงห์,กรรมการ/ผู้ถือหุ้น,0103512009670,ห้างหุ้นส่วนจำกัด เอ๊กโคคอนสตรั๊กชั่น อินยิเนี...,ECHO CONSTRUCTION ENGINEERING LIMITED PARTNERSHIP,ไม่มี,พรวิไลพันธุสิงห์,NaN,99.80
1,จาตุรงค์ จรัสตระกูล,กรรมการ/ผู้ถือหุ้น,0103517003998,ห้างหุ้นส่วนจำกัด จัตุรงค์คูลลิ่ง,CHATURONG COOLING LIMITED PARTNERSHIP,ไม่มี,จาตุรงค์จรัสตระกูล,NaN,93.41
2,ไชย เตชะสมบูรณากิจ,กรรมการ/ผู้ถือหุ้น,0103521001856,ห้างหุ้นส่วนจำกัด โรงงานประดิษฐ์กรณ์เท็กซ์ไทล์,P.D.K.TEXTILE FACTORY LIMITED PARTNERSHIP,การทำนิติกรรมใด ๆ เพื่อให้มีผลผูกพันห้างหุ้นส่...,ไชยเตชะสมบูรณากิจ,มีอำนาจลงนาม,40.00
3,จุรี เตชะสมบูรณากิจ,กรรมการ,0103521001856,ห้างหุ้นส่วนจำกัด โรงงานประดิษฐ์กรณ์เท็กซ์ไทล์,P.D.K.TEXTILE FACTORY LIMITED PARTNERSHIP,การทำนิติกรรมใด ๆ เพื่อให้มีผลผูกพันห้างหุ้นส่...,จุรีเตชะสมบูรณากิจ,มีอำนาจลงนาม,NaN
4,ชาตรี เตชะสมบูรณากิจ,กรรมการ/ผู้ถือหุ้น,0103521001856,ห้างหุ้นส่วนจำกัด โรงงานประดิษฐ์กรณ์เท็กซ์ไทล์,P.D.K.TEXTILE FACTORY LIMITED PARTNERSHIP,การทำนิติกรรมใด ๆ เพื่อให้มีผลผูกพันห้างหุ้นส่...,ชาตรีเตชะสมบูรณากิจ,มีอำนาจลงนาม,30.00


## เลือก File ENLITE ในขอบเขต

In [66]:
df_pm[df_pm['รหัสลูกค้า'] == '0135005']

,รหัสลูกค้า,CUCTAXCS
3728,0135005,0145562001623


In [67]:
df_e1 = pd.merge(df_en2 , df_out4 , left_on = 'เลขทะเบียนนิติบุคคล (13 หลัก)' , right_on = 'CUCTAXCS')

In [68]:
df_enl[df_enl['เลขทะเบียนนิติบุคคล (13 หลัก)'] == '0145562001623']

,ลำดับที่,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,สถานะความเป็นลูกค้า,จังหวัด,สถานภาพ,ชื่อผู้บริหารสูงสุด,อำนาจกรรมการ,ที่อยู่ปัจจุบันแบบเต็ม (Thai),...,รวมส่วนของผู้ถือหุ้น 2564,รวมส่วนของผู้ถือหุ้น 2565,รวมส่วนของผู้ถือหุ้น 2566,จำนวนปีที่ดำเนินการ,วันที่จดทะเบียน,อัตราส่วนความสามารถชำระดอกเบี้ย (เท่า) 2563,อัตราส่วนความสามารถชำระดอกเบี้ย (เท่า) 2564,อัตราส่วนความสามารถชำระดอกเบี้ย (เท่า) 2565,อัตราส่วนความสามารถชำระดอกเบี้ย (เท่า) 2566,ปีงบการเงิน(ล่าสุด)ที่บริษัทนั้น ๆ มี
766,767,0145562001623,บริษัท เซาเทิร์น คิง คอร์ป จำกัด,"SOUTHERN KING CORP CO., LTD.",กลุ่มบริษัททั่วไป,พระนครศรีอยุธยา,ยังดำเนินกิจการอยู่,-,กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,437 หมู่ 3 ตำบล บ้านเลน อำเภอ บางปะอิน จังห...,...,3356378.83,5683343.09,-,5,2562-03-28 00:00:00,-,5.69,4.93,-,2565


In [69]:
df_en2[df_en2['เลขทะเบียนนิติบุคคล (13 หลัก)'] == '0145562001623']

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น
8256,อัจฉรา วฤทธิกุล,กรรมการ/ผู้ถือหุ้น,0145562001623,บริษัท เซาเทิร์น คิง คอร์ป จำกัด,"SOUTHERN KING CORP CO., LTD.",กรรมการหนึ่งคนลงลายมือชื่อและประทับตราสำคัญของ...,อัจฉราวฤทธิกุล,มีอำนาจลงนาม,90.0


In [70]:
print(len(df_e1) , len(df_en1))

4386 16911


In [71]:
def is_debt(a):
  c = 0
  if pd.isna(a) == False:
    if ('หนี้' in a) or ('เงิน' in a):
      c = 1
  return c

In [72]:
df_e1['is_debt'] = df_e1['ผู้ดูแลลูกค้า_(ตามฝ่าย)'].apply(lambda x : is_debt(x))

In [73]:
df_e2 = df_e1[df_e1['is_debt'] == 0].drop(columns = ['is_debt'])

## Clean File รวมศูนย์

In [74]:
df_cen['NAME_C1'] , df_cen['NAME_C2'] =  zip(* df_cen.apply(lambda x : clean_ecs_name( x['NAMETH'] , x['SURNAMETH']) , axis = 1 ))

In [75]:
df_cen['NAME_C3'] , df_cen['NAME_C4'] =  zip(* df_cen.apply(lambda x : clean_ecs_name( x['NAMEEN'] , x['SURNAMEEN']) , axis = 1 ))

In [76]:
name_col = ['NAME_C1' , 'NAME_C2' , 'NAME_C3' , 'NAME_C4']
for i in range(len(name_col)):
  df_cen[name_col[i]] = df_cen[name_col[i]].apply(lambda x : clean_string(x))

In [77]:
df_cen.head()

,EXIM_CUSTID,CUSTID,ID,TYPENAME,TITLENAME,NAMETH,SURNAMETH,NAMEEN,SURNAMEEN,CUSTTYPE,...,WARNINGLIST,CREATEDON,MODIFIEDON,APPROVEDDATE,SYSTEMNAME,EXIM_TITLENAMECODENAME,NAME_C1,NAME_C2,NAME_C3,NAME_C4
0,0058283,6210497,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,11/16/2023 12:50:47 PM,1/15/2024 3:47:20 PM,11/16/2023 2:17:14 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
1,0062173,6205894,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,"ลูกค้า,",...,ทุกครั้ง,3/22/2022 1:52:14 PM,6/9/2023 6:42:38 AM,3/22/2022 1:54:21 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
2,0066484,6208665,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,3/22/2022 2:08:57 PM,1/15/2024 3:48:34 PM,3/22/2022 3:25:46 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
3,6316605,6316605,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,3/13/2024 3:26:22 PM,3/13/2024 5:22:45 PM,3/13/2024 5:50:24 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
4,0140569,6542384,NaN,ข้อมูลเดิม,NaN,ธนาคารอาคารสงเคราะห์แม่โขง,NaN,Mekong Housing Bank,NaN,"ลูกค้า,",...,ทุกครั้ง,12/27/2023 1:22:39 PM,12/27/2023 2:01:32 PM,12/27/2023 2:03:19 PM,LOS,NaN,ธนาคารอาคารสงเคราะห์แม่โขง,NaN,mekonghousingbank,NaN


## เริ่มตรวจทีละรายลูกค้า

In [78]:
df_e1.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),is_in,วันที่อนุมัติครั้งแรก,วันที่อนุมัติล่าสุด,วันที่ทบทวนล่าสุด,CUCTAXCS,is_debt
0,จาตุรงค์ จรัสตระกูล,กรรมการ/ผู้ถือหุ้น,0103517003998,ห้างหุ้นส่วนจำกัด จัตุรงค์คูลลิ่ง,CHATURONG COOLING LIMITED PARTNERSHIP,ไม่มี,จาตุรงค์จรัสตระกูล,NaN,93.41,0103236,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 5,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,1,2018-05-18,2022-06-28,2024-02-01,0103517003998,0
1,วิโรจน์ กมลวิศิษฏ์,กรรมการ/ผู้ถือหุ้น,0103525021479,ห้างหุ้นส่วนจำกัด โกลด์วอเตอร์ส,GOLDWATERS LIMITED PARTNERSHIP,ไม่มี,วิโรจน์กมลวิศิษฎ์,NaN,33.33,0108463,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 5,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,1,2018-11-30,2021-05-05,2024-02-28,0103525021479,0
2,อัมพร กมลวิศิษฎ์ (66.67%),ผู้ถือหุ้น,0103525021479,ห้างหุ้นส่วนจำกัด โกลด์วอเตอร์ส,GOLDWATERS LIMITED PARTNERSHIP,ไม่มี,อัมพรกมลวิศิษฎ์,NaN,66.67,0108463,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 5,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,1,2018-11-30,2021-05-05,2024-02-28,0103525021479,0
3,สุทัศน์ เข็มจินดา,กรรมการ/ผู้ถือหุ้น,0103531005731,ห้างหุ้นส่วนจำกัด ทรรศพร อินเตอร์เนชั่นแนล เทรด,TASAPORN INTERNATIONAL TRADE LIMITED PARTNERSHIP,ไม่มี,สุทัศน์เข็มจินดา,NaN,80.00,0138794,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 2,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,2,2024-01-18,2024-03-27,2024-02-28,0103531005731,0
4,นพพร เข็มจินดา (20.00%),ผู้ถือหุ้น,0103531005731,ห้างหุ้นส่วนจำกัด ทรรศพร อินเตอร์เนชั่นแนล เทรด,TASAPORN INTERNATIONAL TRADE LIMITED PARTNERSHIP,ไม่มี,นพพรเข็มจินดา,NaN,20.00,0138794,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 2,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,2,2024-01-18,2024-03-27,2024-02-28,0103531005731,0


In [79]:
df_cen.head()

,EXIM_CUSTID,CUSTID,ID,TYPENAME,TITLENAME,NAMETH,SURNAMETH,NAMEEN,SURNAMEEN,CUSTTYPE,...,WARNINGLIST,CREATEDON,MODIFIEDON,APPROVEDDATE,SYSTEMNAME,EXIM_TITLENAMECODENAME,NAME_C1,NAME_C2,NAME_C3,NAME_C4
0,0058283,6210497,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,11/16/2023 12:50:47 PM,1/15/2024 3:47:20 PM,11/16/2023 2:17:14 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
1,0062173,6205894,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,"ลูกค้า,",...,ทุกครั้ง,3/22/2022 1:52:14 PM,6/9/2023 6:42:38 AM,3/22/2022 1:54:21 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
2,0066484,6208665,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,3/22/2022 2:08:57 PM,1/15/2024 3:48:34 PM,3/22/2022 3:25:46 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
3,6316605,6316605,NaN,ข้อมูลเดิม,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,PETROLEUM AUTHORITY OF THAILAND,NaN,ผู้ถือหุ้นตั้งแต่ 20% ขึ้นไป/ ผู้รับผลประโยชน์...,...,ทุกครั้ง,3/13/2024 3:26:22 PM,3/13/2024 5:22:45 PM,3/13/2024 5:50:24 PM,LOS,NaN,การปิโตรเลียมแห่งประเทศไทย,NaN,petroleumauthorityofthailand,NaN
4,0140569,6542384,NaN,ข้อมูลเดิม,NaN,ธนาคารอาคารสงเคราะห์แม่โขง,NaN,Mekong Housing Bank,NaN,"ลูกค้า,",...,ทุกครั้ง,12/27/2023 1:22:39 PM,12/27/2023 2:01:32 PM,12/27/2023 2:03:19 PM,LOS,NaN,ธนาคารอาคารสงเคราะห์แม่โขง,NaN,mekonghousingbank,NaN


In [80]:
cus_list = df_e2['รหัสลูกค้า'].unique().tolist()
name_centralize = ['NAME_C1' , 'NAME_C2' ,'NAME_C3' , 'NAME_C4']
df_notin = pd.DataFrame()
for i in range(len(cus_list)):
  df_u = df_e2[df_e2['รหัสลูกค้า'] == cus_list[i]]
  df_c = df_cen[df_cen['EXIM_CUSTID'] == cus_list[i]]
  df_m_out = df_u.copy()
  for n in range(len(name_centralize)):
    if (len(df_m_out.dropna(subset = ['Name_clean'])) != 0) and len(df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates()):
      df_m = fuzzy_merge(df_m_out.dropna(subset = ['Name_clean']) ,
                                    df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates() ,
                                    left_on = 'Name_clean' , right_on = name_centralize[n] , how = 'left')
      df_m_out = df_m[df_m[name_centralize[n]].isna()].drop(columns = name_centralize[n])
  df_notin = pd.concat([df_notin , df_m_out])

In [81]:
len(df_notin)

306

In [82]:
def id_g(a):
  c = 0
  if pd.isna(a) == False:
    if a[0] == 'G':
      c = 1
  return c

In [83]:
df_notin['is_g'] = df_notin['รหัสลูกค้า'].apply(lambda x : id_g(x))

In [84]:
df_g = df_notin[df_notin['is_g'] == 1].drop(columns = ['is_g'])

df_notg = df_notin[df_notin['is_g'] == 0].drop(columns = ['is_g'])

In [85]:
print(len(df_g) , len(df_notg))

48 258


In [86]:
cus_list = df_g['รหัสลูกค้า'].unique().tolist()
name_centralize = ['NAME_C1' , 'NAME_C2' ,'NAME_C3' , 'NAME_C4']
df_notin1 = pd.DataFrame()
for i in range(len(cus_list)):
  df_u = df_g[df_g['รหัสลูกค้า'] == cus_list[i]]
  df_c = df_cen.copy()
  df_m_out = df_u.copy()
  for n in range(len(name_centralize)):
    if (len(df_m_out.dropna(subset = ['Name_clean'])) != 0) and len(df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates()):
      df_m = fuzzy_merge(df_m_out.dropna(subset = ['Name_clean']) ,
                                    df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates() ,
                                    left_on = 'Name_clean' , right_on = name_centralize[n] , how = 'left')
      df_m_out = df_m[df_m[name_centralize[n]].isna()].drop(columns = name_centralize[n])
  df_notin1 = pd.concat([df_notin1 , df_m_out])

In [87]:
df_ans = pd.concat([df_notg , df_notin1])

In [88]:
df_ans.head()

,ชื่อผู้เกี่ยวข้อง,ความเกี่ยวข้อง,เลขทะเบียนนิติบุคคล (13 หลัก),ชื่อนิติบุคคลจดทะเบียนภาษาไทย,ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ,อำนาจกรรมการ,Name_clean,อำนาจลงนาม,เปอร์เซ็นผู้ถือหุ้น,รหัสลูกค้า,ผู้ดูแลลูกค้า_(แยกส่วน),ผู้ดูแลลูกค้า_(ตามฝ่าย),is_in,วันที่อนุมัติครั้งแรก,วันที่อนุมัติล่าสุด,วันที่ทบทวนล่าสุด,CUCTAXCS
0,ภาคิน บุญญเบญจฤทธิ์,กรรมการ/ผู้ถือหุ้น,0103535014050,ห้างหุ้นส่วนจำกัด เรืองทรัพย์ เทรดดิ้ง,RUANGSARP TRADING LIMITED PARTNERSHIP,ในการดำเนินการใดๆ ให้มีผลผูกพันห้างหุ้นส่วนให้...,ภาคินบุญญเบญจฤทธิ์,มีอำนาจลงนาม,20.00,0110299,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 1,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,1,2024-01-18,2024-01-18,2022-04-22,0103535014050
1,ณัชชา บุญญเบญจฤทธิ์,กรรมการ/ผู้ถือหุ้น,0103535014050,ห้างหุ้นส่วนจำกัด เรืองทรัพย์ เทรดดิ้ง,RUANGSARP TRADING LIMITED PARTNERSHIP,ในการดำเนินการใดๆ ให้มีผลผูกพันห้างหุ้นส่วนให้...,ณัชชาบุญญเบญจฤทธิ์,มีอำนาจลงนาม,20.00,0110299,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 1,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,1,2024-01-18,2024-01-18,2022-04-22,0103535014050
0,วารุณี อดิเรก,กรรมการ/ผู้ถือหุ้น,0103537001419,ห้างหุ้นส่วนจำกัด ใบโพธิ์ธุรกิจและอุตสาหกรรม,BAIPO BUSINESS AND INDUSTRY LIMITED PARTNERSHIP,ไม่มี,วารุณีอดิเรก,NaN,30.00,0143703,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 6,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,1,2024-01-18,2024-01-18,NaT,0103537001419
1,วิทวัส อดิเรก (25.00%),ผู้ถือหุ้น,0103537001419,ห้างหุ้นส่วนจำกัด ใบโพธิ์ธุรกิจและอุตสาหกรรม,BAIPO BUSINESS AND INDUSTRY LIMITED PARTNERSHIP,ไม่มี,วิทวัสอดิเรก,NaN,25.00,0143703,ส่วนการตลาดวิสาหกิจขนาดกลางและขนาดย่อม 6,ฝ่ายการตลาดวิสาหกิจขนาดกลางและขนาดย่อม,1,2024-01-18,2024-01-18,NaT,0103537001419
0,อภิชาต ตั้งอมรศิริ (21.69%),ผู้ถือหุ้น,0105508000231,บริษัท ที.พี.ดรัก แลบบอราทอรี่ส์ (1969) จำกัด,"T.P.DRUGLABORATORIES (1969) CO.,LTD.",นายอุดมชัย ตั้งอมรศิริ หรือนายอุดมศักดิ์ ตั้งอ...,อภิชาตตั้งอมรศิริ,NaN,21.69,0078106,สาขาบางนา-ตราด กม.3,สาขาบางนา-ตราด กม.3,1,2020-12-21,2023-03-21,2024-03-27,0105508000231


# EXPORT FILE

In [89]:
df_a1 = df_ans[[ 'รหัสลูกค้า' ,  'เลขทะเบียนนิติบุคคล (13 หลัก)' , 'ชื่อนิติบุคคลจดทะเบียนภาษาไทย', 'ชื่อนิติบุคคลจดทะเบียนภาษาอังกฤษ' ,
                      'ชื่อผู้เกี่ยวข้อง' ,'ความเกี่ยวข้อง',  'อำนาจกรรมการ', 'อำนาจลงนาม', 'เปอร์เซ็นผู้ถือหุ้น' , 'ผู้ดูแลลูกค้า_(แยกส่วน)', 'ผู้ดูแลลูกค้า_(ตามฝ่าย)',
                        'วันที่อนุมัติครั้งแรก', 'วันที่อนุมัติล่าสุด', 'วันที่ทบทวนล่าสุด' , 'is_in']]

In [90]:
df_a1.to_excel(result_path + file_name , index = False)

In [91]:
def clean_date_out(a):
  if (pd.isna(a) == False) and (a[-4:-2] != '20') and (a[-4:-2] != '19'):
    a = a.replace('/265' , '/256')
    return  a[:-4] + str(int(a[-4:]) - 543)
  elif (pd.isna(a) == False) and (a[:2] != '00'):
    a = np.nan
    return a
  else:
    return a